<a href="https://colab.research.google.com/github/holehouse-lab/EnsPredict-colab/blob/main/polymer_property_predictors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to IDPEnsemble Predictor

In [3]:
#@title Install Necessary Dependencies
!pip install git+https://git@github.com/idptools/sparrow.git@polymer_predictors --quiet




  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.6/142.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.7/476.7 kB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.0 MB/s eta 0:00:00


In [4]:
#@title File Imports
import sparrow
from sparrow.predictors.batch_predict import batch_predict
from google.colab import data_table
import pandas as pd
from sparrow.sparrow_exceptions import SparrowException
from google.colab import files
import io
from collections import defaultdict

Error importing GPy.
 If trying to run parrot-optimize, make sure to use `pip install idptools-parrot[optimize]`


In [6]:
#@title Target Sequence
#@markdown <h1>Enter the primary amino acid sequence:</h1>

#@markdown Notes: 

#@markdown - The following predictors implicitly assume this sequence is disordered.

#@markdown - A sequence cannot be provided at the same time as a fasta file.

# define the function that will be called when the form is submitted
def process_form():
    uploaded_data = files.upload()
    uploaded_fasta = list(uploaded_data.keys())[0]
    # Load protein objects
    try:
      protein_objs = sparrow.read_fasta(uploaded_fasta)
    except TypeError as e:
      print(f"Received TypeError: Perhaps you didn't upload a file?")
      raise e

    return protein_objs



sequence = "" #@param {type:"string"}
fasta_file = True #@param {type:"boolean"}


if fasta_file and len(sequence) > 0:
  raise SparrowException(f"Cannot specify both a sequence and a list of sequences from a fasta file simultaneously")

if fasta_file:
    protein_objs = process_form()
else:
    # list to make it an iterable
    protein_objs = [sparrow.Protein(sequence)]

for protein in protein_objs:
  if len(protein) > 750:
    raise SparrowException(f"Sequences longer than 750 residues are not currently supported."\
      "You entered a protein with length {len(protein)} residues")



Saving test.fasta to test.fasta


In [2]:
#@title Sequence Property Prediction

#@markdown <h1>Please select the ensemble features you'd like to predict and then press the run cell button: </h1>

#@markdown This may take some time for large fasta files. GPU batch support is currently not implemented. Power users are recommended to install sparrow to run predictions locally.

radius_of_gyration = True #@param {type:"boolean"}
end_to_end_distance = True #@param {type:"boolean"}
scaling_exponent = True #@param {type:"boolean"}
prefactor = True #@param {type:"boolean"}
asphericity = True #@param {type:"boolean"}


dimensions = defaultdict(dict)

for i, (header,protein) in enumerate(protein_objs.items(),1):
    
    dimensions[header].update({"length" : int(len(protein))})

    if radius_of_gyration:
        dimensions[header].update({"radius_of_gyration" : i*10})
    if end_to_end_distance:
    # dimensions["end_to_end_distance"] = protein.predictor.end_to_end_distance()
        dimensions[header].update({"end_to_end_distance" : 12.5})
    if scaling_exponent:
        dimensions[header].update({"scaling_exponent" : 12.5})
    # dimensions["scaling_exponent"] = protein.predictor.scaling_exponent()
    if asphericity:
        dimensions[header].update({"asphericity" : 5.5})
        # dimensions["asphericity"] = 0.8 #protein.predictor.asphericity()
    if prefactor:
        dimensions[header].update({"prefactor" : 5.5})
        # dimensions["prefactor"] = 5.5 # protein.predictor.prefactor()

df = pd.DataFrame(dimensions).T
df = df.rename_axis('fasta_header')

display(data_table.DataTable(df, include_index=True, num_rows_per_page=5))


NameError: ignored

## Predicted Properties

In [ ]:
# temporarily not including this in notebook but will add later this week.
# seq = sparrow.Protein("GSKPRKGDLRSDPLEDKRKSRGSPKR")
# p_of_Re_R, p_of_Re_P = seq.polymeric.get_predicted_nu_dep_end_to_end_distribution()